IF THIS FEEL USEFUL, PLEASE GOOD. 
### Columns
* battery_power: バッテリーが一度に蓄えることができる総エネルギー（mAhで測定)
*  blue: bluetoothが有/無
*  clock_speed: マイクロプロセッサが命令を実行する速度
*  dual_sim: デュアルシムの有/無
*  fc: フロントカメラのメガピクセル
*  four_g: 4G有/無
*  int_memory: ギガバイト単位の内部メモリ
*  m_dep: モバイルの深さ（cm)
*  mobile_wt: 携帯電話の重量
*  n_cores: プロセッサのコア数

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as ex
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
train = "../input/mobile-price-classification/train.csv"
test = "../input/mobile-price-classification/test.csv"
train_data = pd.read_csv(train)
test_data = pd.read_csv(test)
data = pd.concat([train_data, test_data])

In [ ]:
# データ型やnullを調べた
data.info()

In [ ]:
data.describe()

In [ ]:
#pandas_profilingについて調べてみたら中々クールだったので使ってみた。

#import pandas_profiling as pandas_pf
#pandas_pf.ProfileReport(data)

In [ ]:
# html化
#profile = pandas_pf.ProfileReport(data)
#profile.to_file(outputfile="myoutputfile.html")

In [ ]:
# データ全体の分布
data.hist(figsize = (35,30), bins = 50)
plt.show()

In [ ]:
range_mapping = {0: "cheap",
    1: "standart",
    2: "expensive",
    3: "very expensive"}

In [ ]:
train_data['price_range_mapped'] = train_data['price_range'].map(range_mapping)

In [ ]:
# https://seaborn.pydata.org/generated/seaborn.FacetGrid.html
grid = sns.FacetGrid(train_data, col="price_range", hue="price_range_mapped", palette="hsv",height=5
                   )
grid.map(sns.distplot, "ram").add_legend()

#### 4つのタイプのprice range:

*  Cheap: RAM ranges between 0 to 2000 MB
*  Standart: RAM ranges between 0 to 3000 MB
*  Expensive: RAM ranges between 1000 to 4000 MB
*  Very Expensive: RAM ranges between 2000 to 4000 MB

In [ ]:
# price_range_mapped==cheap 特徴量の割合
cheap_data = train_data.query('price_range_mapped == "cheap"')
cheap_data.T

In [ ]:
cheap_data.describe()

In [ ]:
# 第一引数:dataframe, 第2引数カラム名
fig = ex.pie(cheap_data,names='four_g')
# タイトル付与
fig.update_layout(title='4Gの割合')
fig.show()
# 割合を表にしたい

In [ ]:
unique_col = []
for e in cheap_data.columns:
    unique_e = cheap_data[e].unique().tolist()
    sort_unique = sorted(unique_e)
    if sort_unique == [0,1]:
        # print(f"{e}:{sort_unique}")
        unique_col.append(e)

# print(unique_col)
cheap_data[unique_col].head()

In [ ]:
cheap_data[unique_col].rename(columns={'blue':'bluetooth'})

In [ ]:
# extract columns without 1 or 0 
not_unique_col = list(set(cheap_data.columns) - set(unique_col))

In [ ]:
# hist not_unique_col
cheap_data[not_unique_col].hist(figsize = (35,30), bins = 50)
plt.show()

In [ ]:
cheap_data[not_unique_col]

In [ ]:
features = ['battery_power', 'blue', 'clock_speed','dual_sim', 'fc', 'four_g', 'int_memory', 'm_dep', 'mobile_wt', 'n_cores',
            'pc', 'px_height', 'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g', 'touch_screen', 'wifi']
target = ['price_range']

In [ ]:
x = train_data[features].values
y = train_data[target].values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state = 24)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_predict = lr.predict(X_test)
lr_conf_matrix = confusion_matrix(y_test, lr_predict)
lr_acc_score = accuracy_score(y_test, lr_predict)
print("confussion matrix")
print(lr_conf_matrix)
print("-------------------------------------------")
print("Accuracy of Logistic Regression:",lr_acc_score*100,'\n')
print("-------------------------------------------")

EDA IS CONTINUED...